<a href="https://colab.research.google.com/github/shannonshih/Tibame_GAD245-Practice-HW_0319/blob/main/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

安裝所需模組

In [1]:
!pip install protobuf --quiet
!pip install numpy>=1.26.0 --quiet
!pip install apache_beam==2.60.0 --quiet
!pip install tensorflow==2.12.0 --quiet
# To use the newly installed versions, restart the runtime.
exit()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

授予GCP訪問權限

In [1]:
from google.colab import auth
auth.authenticate_user()

導入模組並定義常數

In [3]:
import argparse
from typing import Dict, Text, Any, Tuple, List
import numpy

from google.protobuf import text_format

import tensorflow as tf
from tensorflow import keras
import apache_beam as beam
from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.base import KeyedModelHandler
from apache_beam.ml.inference.tensorflow_inference import TFModelHandlerNumpy
from apache_beam.ml.inference.tensorflow_inference import TFModelHandlerTensor
from apache_beam.options.pipeline_options import PipelineOptions

project = "tibame-gad245-16-0319"
bucket = "tibame-gad245-16-0319"

save_model_dir_multiply = f'gs://{bucket}/tf-inference/model/multiply_five/v1/'
save_weights_dir_multiply = f'gs://{bucket}/tf-inference/weights/multiply_five/v1/'

建立線性回歸模型

In [4]:
# Create training data that represents the 5 times multiplication table for the numbers 0 to 99.
# x is the data and y is the labels.
x = numpy.arange(0, 100)   # Examples
y = x * 5                  # Labels

# Use create_model to build a simple linear regression model.
# Note that the model has a shape of (1) for its input layer and expects a single int64 value.
def create_model():
  input_layer = keras.layers.Input(shape=(1), dtype=tf.float32, name='x')
  output_layer= keras.layers.Dense(1)(input_layer)
  model = keras.Model(input_layer, output_layer)
  model.compile(optimizer=tf.optimizers.Adam(), loss='mean_absolute_error')
  return model

model = create_model()
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 x (InputLayer)              [(None, 1)]               0         
                                                                 
 dense (Dense)               (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


執行FIT

In [5]:
model.fit(x, y, epochs=500, verbose=0)
test_examples =[20, 40, 60, 90]
value_to_predict = numpy.array(test_examples, dtype=numpy.float32)
predictions = model.predict(value_to_predict)

print('Test Examples ' + str(test_examples))
print('Predictions ' + str(predictions))

1/1 [==============================] - 0s 69ms/step
Test Examples [20, 40, 60, 90]
Predictions [[ 49.79167]
 [ 97.58705]
 [145.38243]
 [217.07552]]


儲存模型

In [8]:
model.save(save_model_dir_multiply)
#model_handler = TFModelHandlerNumpy(path_to_weights, model_type=ModelType.SAVED_WEIGHTS, create_model_fn=build_tensorflow_model)
model.save_weights(save_weights_dir_multiply)

訓練模型預測

In [9]:
class FormatOutput(beam.DoFn):
  def process(self, element, *args, **kwargs):
     """格式化推論結果輸出。"""
     yield "範例值為 {example}，預測結果為 {prediction}".format(example=element.example, prediction=element.inference)

# 建立輸入範例數據
examples = numpy.array([20, 40, 60, 90], dtype=numpy.float32)

# 設定 TensorFlow 模型處理器
model_handler = TFModelHandlerNumpy(save_model_dir_multiply)

with beam.Pipeline() as p:
    _ = (p
         | "建立輸入數據" >> beam.Create(examples)
         | "執行模型推論" >> RunInference(model_handler)
         | "格式化輸出結果" >> beam.ParDo(FormatOutput())
         | "輸出結果" >> beam.Map(print)
        )

範例值為 20.0，預測結果為 [49.79167]
範例值為 40.0，預測結果為 [97.58705]
範例值為 60.0，預測結果為 [145.38243]
範例值為 90.0，預測結果為 [217.07552]


加入權重預測

In [10]:
from apache_beam.ml.inference.tensorflow_inference import ModelType

# 建立輸入範例數據
examples = numpy.array([20, 40, 60, 90], dtype=numpy.float32)

# 設定 TensorFlow 模型處理器，使用儲存的權重進行推論
model_handler = TFModelHandlerNumpy(
    save_weights_dir_multiply,
    model_type=ModelType.SAVED_WEIGHTS,
    create_model_fn=create_model
)

with beam.Pipeline() as p:
    _ = (p
         | "建立輸入數據" >> beam.Create(examples)
         | "執行模型推論" >> RunInference(model_handler)
         | "格式化輸出結果" >> beam.ParDo(FormatOutput())
         | "輸出結果" >> beam.Map(print)
        )

範例值為 20.0，預測結果為 [49.79167]
範例值為 40.0，預測結果為 [97.58705]
範例值為 60.0，預測結果為 [145.38243]
範例值為 90.0，預測結果為 [217.07552]


鍵值資料預測

In [11]:
class FormatOutputKeyed(FormatOutput):
  # 繼承自 FormatOutput。
  def process(self, tuple_in: Tuple):
    key, element = tuple_in
    output = super().process(element)
    yield "{} : {}".format(key, [op for op in output])

# 建立輸入範例數據（包含鍵值）
examples = numpy.array([(1, 20), (2, 40), (3, 60), (4, 90)], dtype=numpy.float32)

# 設定鍵值模型處理器
keyed_model_handler = KeyedModelHandler(TFModelHandlerNumpy(save_model_dir_multiply))

with beam.Pipeline() as p:
    _ = (p
         | "建立輸入數據" >> beam.Create(examples)
         | "執行模型推論" >> RunInference(keyed_model_handler)
         | "格式化輸出結果（含鍵值）" >> beam.ParDo(FormatOutputKeyed())
         | "輸出結果" >> beam.Map(print)
        )

1.0 : ['範例值為 20.0，預測結果為 [49.79167]']
2.0 : ['範例值為 40.0，預測結果為 [97.58705]']
3.0 : ['範例值為 60.0，預測結果為 [145.38243]']
4.0 : ['範例值為 90.0，預測結果為 [217.07552]']
